# **Load Data into ES - Test-1**

In [1]:
import os
import csv
import json
import pandas as pd
import numpy as np


In [92]:
# ------------------------- Create an ES Client -------------------------
from elasticsearch import Elasticsearch
es_client = Elasticsearch(
    "localhost:9200",
    http_auth=["elastic", "changeme"], 
) 
# ------------------------- Create an ES Index Client -------------------------
from elasticsearch.client import IndicesClient
es_index_client = IndicesClient(es_client)
type(es_index_client)

elasticsearch.client.indices.IndicesClient

In [93]:
# ------------------------- Define the Settings & Mappings - Ver.1 -------------------------
configurations = {
  "settings": {
      "index": {
          "number_of_replicas": 1},
      "analysis": {
          "filter": {
              "ngram_filter": {
                "type": "edge_ngram",
                "min_gram": 2,
                "max_gram": 50}
          },
          "analyzer": {
              "ngram_analyzer": {
                  "type": "custom",
                  "tokenizer": "standard",
                  "filter": [
                    "lowercase",
                    "ngram_filter"]
              }
          }
      }
  },
  "mappings": {
        "properties": {
            "id": {
                "type": "long"},
            "label": {
                "type": "long"},
            "company": {
                "type": "text"},
            "industry": {
                "type": "text"},
            "country": {
                "type": "text"},
            "date": {
                "type": "text"},
            "content": {
                "type": "text",
                "analyzer": "standard",
                "fields": {
                    "keyword": {
                        "type": "keyword"},
                    "ngrams": {
                        "type": "text",
                        "analyzer": "ngram_analyzer"}
                }
                # "type": "nested",
                # "properties": {
                    #-------------------
                    # "attribute_name": {
                    #     "type": "text"},
                    # "attribute_value": {
                    #     "type": "text"}
                    #-------------------
                    # "page": {
                    #     "type": "long"},
                    # "priority": {
                    #     "type": "float"},
                    # "sentence": {
                    #     "type": "text",
                    #     "analyzer": "standard",
                    #     "fields": {
                    #         "keyword": {
                    #             "type": "keyword"},
                    #         "ngrams": {
                    #             "type": "text",
                    #             "analyzer": "ngram_analyzer"}
                    #     }
                    # }
                    #-------------------
                # }
            }
        }
    }
}

# ------------------------- Create an ES Index -------------------------
es_index_client.create(index="esg_report_1", body=configurations)

/var/folders/cb/3qy8wy6s2571t48bzrjk5vl80000gn/T/ipykernel_38909/3029866595.py:79: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_index_client.create(index="esg_report_1", body=configurations)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'esg_report_1'}

In [ ]:
# Reset Index with new configurations
# in Kibana:    DELETE esg_report_1 

In [4]:
# List all trimmed CSV files from the Crawler
pos_file_list = os.listdir("Crawler & Processing/2.Develop - Crawler Folder/preprocessed/trimmed/pos")
pos_file_list.sort()
print("Total number of positive CSV files:", len(pos_file_list))
print("Type of pos_file_list items:", type(pos_file_list[0]))
# print("Show the pos_file_list:", pos_file_list)

neg_file_list = os.listdir("Crawler & Processing/2.Develop - Crawler Folder/preprocessed/trimmed/neg")
neg_file_list.sort()
print("Total number of negative CSV files:", len(neg_file_list))
print("Type of pos_file_list items:", type(neg_file_list[0]))
# print("Show the pos_file_list:", neg_file_list)

Total number of positive CSV files: 196
Type of pos_file_list items: <class 'str'>
Total number of negative CSV files: 177
Type of pos_file_list items: <class 'str'>


In [6]:
# Load companies.xlsx for company info
com_details = pd.read_excel("Crawler & Processing/2.Develop - Crawler Folder/companies.xlsx")

# Display df info
com_details.head()

,Unique ID,Issuer - subsidiary,Issuer industry,Country of risk,Date
0,4047,Beijing Enterprises Water Capital Management H...,Utilities,China,2021-04-30
1,16478,Nanjing Financial City Construction & Developm...,Financials,China,2021-04-30
2,16481,Suzhou Tech City Development Group Co Ltd,Financials,China,2021-04-30
3,16479,Landesbank Baden-Wuerttemberg,Financials,Germany,2021-04-30
4,16480,City of Lunds Sweden,Government,Sweden,2021-04-30


In [22]:
####### Create df for Positive Reports listed in pos_file_list
column_names = ["id", "label", "company", "industry", "country", "date"]

pos_df = pd.DataFrame(columns = column_names)

for file in pos_file_list:
    # Get the file name without ".csv"
    pdf_ID = int(file.split(".")[0])
    # Positive label defined as 1
    pos_label = 1
    # Find matched rows for each pdf_ID and get the first row to match "trimmed" csv files
    matched_row = com_details[com_details["Unique ID"]==pdf_ID].iloc[0] 
    company_name = matched_row["Issuer - subsidiary"]
    industry_name = matched_row["Issuer industry"]
    country_name = matched_row["Country of risk"]
    date_str = str(matched_row["Date"]).split(" ")[0]
    
    pos_df = pos_df.append({"id": pdf_ID, 
                            "label": pos_label, 
                            "company": company_name, 
                            "industry": industry_name, 
                            "country": country_name,
                            "date": date_str}, ignore_index=True)

print("Check out pos_df:")
display(pos_df.head())
print("Shape of pos_df:", pos_df.shape)


Check out pos_df:


,id,label,company,industry,country,date
0,11,1,Aurubis AG,Materials,Germany,2020-06-10
1,12,1,Lenzing AG,Materials,Austria,2020-01-10
2,13494,1,Liberty Utilities Finance GP 1,Utilities,Canada,2020-09-16
3,13498,1,Falck Renewables SpA,Utilities,Italy,2020-09-16
4,13770,1,Union Electric Co,Utilities,United States,2020-10-01


Shape of pos_df: (196, 6)


In [31]:
####### Create df for Negative Reports listed in neg_file_list
column_names = ["id", "label", "company", "industry", "country", "date"]

neg_df = pd.DataFrame(columns = column_names)

for file in neg_file_list:
    # Get the file name without ".csv"
    pdf_ID = int(file.split("_")[0])
    # Positive label defined as -1
    pos_label = -1
    # Find matched rows for each pdf_ID and get the first row to match "trimmed" csv files
    matched_row = com_details[com_details["Unique ID"]==pdf_ID].iloc[0] 
    company_name = matched_row["Issuer - subsidiary"]
    industry_name = matched_row["Issuer industry"]
    country_name = matched_row["Country of risk"]
    date_str = str(file.split(".")[0]).split("_")[1] # Use the date contained in file name
    
    neg_df = neg_df.append({"id": pdf_ID, 
                            "label": pos_label, 
                            "company": company_name, 
                            "industry": industry_name, 
                            "country": country_name,
                            "date": date_str}, ignore_index=True)

print("Check out neg_df:")
display(neg_df.head())
print("Shape of neg_df:", neg_df.shape)


Check out neg_df:


,id,label,company,industry,country,date
0,110,-1,Mercon BV,Consumer staples,Netherlands,2018
1,112,-1,Nokian Renkaat Oyj,Consumer discretionary,Finland,2018
2,11,-1,Aurubis AG,Materials,Germany,2019
3,124,-1,Vinte Viviendas Integrales SAB de CV,Consumer discretionary,Mexico,2018
4,127,-1,Obayashi Corp,Industrials,Japan,2018


Shape of neg_df: (177, 6)


In [95]:
######## Bulk Process Positive Reports
columns = ["page", "priority", "content"]
index_name = "esg_report_1"
action_list = []  

for fileName in pos_file_list:
    pdf_ID = int(fileName.split(".")[0])
    matched_row = pos_df[pos_df["id"]==pdf_ID].iloc[0] 
    
    # action = {"index": {"_index": index_name, "_id": pdf_ID}}  
    # doc = {
    #         "id": pdf_ID, 
    #         "label": matched_row["label"],
    #         "company": matched_row["company"],
    #         "industry": matched_row["industry"],
    #         "country": matched_row["country"],
    #         "date": matched_row["date"],
    #         }
    # action_list.append(json.dumps(action))  
    # action_list.append(json.dumps(doc))
    
    
    
    # with open("Crawler & Processing/2.Develop - Crawler Folder/preprocessed/trimmed/pos/" + fileName, "r") as f:
    #     csv_file = csv.DictReader(f, fieldnames=columns, delimiter=",", quotechar='"') 
    #     next(csv_file)
        
    # action = {"index": {"_index": index_name, "_id": pdf_ID}}  
    # doc = {
    #         "id": pdf_ID, 
    #         "label": matched_row["label"],
    #         "company": matched_row["company"],
    #         "industry": matched_row["industry"],
    #         "country": matched_row["country"],
    #         "date": matched_row["date"],
    #         "content": [
    #             for row in csv_file:
    #                 {"page": row["page"]},
    #                 {"priority": row["priority"]},
    #                 {"sentence": row["content"]}]
    #         }
    # action_list.append(json.dumps(action))  
    # action_list.append(json.dumps(doc))
    
    ###### Concacte all text into one string per doc
    content_str = ""
    with open("Crawler & Processing/2.Develop - Crawler Folder/preprocessed/trimmed/pos/" + fileName, "r") as f:
        csv_file = csv.DictReader(f, fieldnames=columns, delimiter=",", quotechar='"')
        next(csv_file)  
        for row in csv_file:
            content_str += row["content"]
        

    action = {"index": {"_index": index_name, "_id": pdf_ID}} 
    doc = {
            "id": pdf_ID, 
            "label": matched_row["label"],
            "company": matched_row["company"],
            "industry": matched_row["industry"],
            "country": matched_row["country"],
            "date": matched_row["date"],
            "content": content_str
                        # [
                        # #-------------------
                        # # {"page": row["page"]},
                        # # {"priority": row["priority"]},
                        # # {"sentence": row["content"]}
                        # #-------------------
                        # # {"attribute_name": "page", "attribute_value": row["page"]},
                        # # {"attribute_name": "priority", "attribute_value": row["priority"]},
                        # # {"attribute_name": "text", "attribute_value": row["content"]}
                        # ]
            }
    action_list.append(json.dumps(action))  
    action_list.append(json.dumps(doc)) 

# ------------------------- Feed the data into a JSON file -------------------------
with open("esg_report_1.json", "w") as write_file:
    write_file.write("\n".join(action_list))
    

    

In [96]:
# ---------------------- Feed JSON File to ES - Bulk Upload!!! ----------------------
es_client.bulk(body="\n".join(action_list))

{'took': 1303,
 'errors': True,
 'items': [{'index': {'_index': 'esg_report_1',
    '_type': '_doc',
    '_id': '11',
    'status': 400,
    'error': {'type': 'illegal_argument_exception',
     'reason': 'Document contains at least one immense term in field="content.keyword" (whose UTF8 encoding is longer than the max length 32766), all of which were skipped.  Please correct the analyzer to not produce such terms.  The prefix of the first immense term is: \'[75, 101, 121, 32, 109, 101, 97, 115, 117, 114, 101, 115, 32, 111, 102, 32, 116, 104, 101, 32, 83, 117, 115, 116, 97, 105, 110, 97, 98, 105]...\', original message: bytes can be at most 32766 in length; got 32793',
     'caused_by': {'type': 'max_bytes_length_exceeded_exception',
      'reason': 'bytes can be at most 32766 in length; got 32793'}}}},
  {'index': {'_index': 'esg_report_1',
    '_type': '_doc',
    '_id': '12',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_